In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [2]:
from numpy.random import seed
seed(42)
from tensorflow import random
random.set_seed(42)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [3]:
data = pd.read_csv('Data-SMS.csv', encoding='utf-8')

In [239]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Class   400 non-null    object
 1   Text    400 non-null    object
dtypes: object(2)
memory usage: 6.4+ KB


In [240]:
data = data.rename(columns={'Class': 'label', 'Text': 'text'})
data['label'] = data['label'].map({'real': 0, 'defraud': 1})

In [241]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   400 non-null    int64 
 1   text    400 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


In [242]:
data

,label,text
0,1,รับเลขมงคล และผลสลากทุกงวด 5บ/ข้อความ สมัคร *3...
1,1,ดวงเป๊ะปัง รับทุกวัน เฮง ๆ รวย ๆ 5บ/ข้อความ สม...
2,1,ฟรีเด้อ! เว้าบาป-ตรี ชัยณรงค์ แถมฟรีค่าบริการเ...
3,1,ฟรี! เพลงรอสาย บุคคลทั่วไป-ลำเพลิน วงศกร แถมฟร...
4,1,10.10 โหลดฟรี! เพลงรอสาย เวลา-ป๊อบ ปองกูล แถมฟ...
...,...,...
395,0,"แพ็กที่ซื้อ 23/10/22 ใกล้จะหมดอายุแล้ว, รับฟรี..."
396,0,อย่าลืม! คุณมีเงินคืน 16.00บ.จะหมดอายุเวลา 09:...
397,0,คุณได้เติมเงินจำนวน 50บ.ด้วยบัตรเงินสดทรูมันนี...
398,0,อย่าลืมจ่ายบิลทรูของคุณ พิเศษเหมือนเคย รับเงิน...


In [243]:
import pythainlp
from nltk.tokenize import word_tokenize
import re
'''''
def tokenize(text):
    thai_words = pythainlp.word_tokenize(text, engine='newmm')
    eng_words = word_tokenize(text)
    words = thai_words + eng_words
    return words

def tokenize(text):
    thai_words = pythainlp.word_tokenize(text, engine='newmm')
    eng_words = word_tokenize(text)
    words = [word.strip() for word in thai_words + eng_words]
    return words
'''''

def tokenize(text):
    text = re.sub(r'\d+', 'num', text)
    thai_words = pythainlp.word_tokenize(text,keep_whitespace=False, engine='newmm')
    eng_words = word_tokenize(text)
    #words = [word.replace(' ','' ) for word in thai_words + eng_words]
    words = thai_words + eng_words
    return words


In [244]:
'''''
import string
import pythainlp
from pythainlp.tokenize import word_tokenize

def tokenize(text, keep_whitespace=False, keep_punctuations=False):
    thai_words = pythainlp.word_tokenize(text, engine='newmm', keep_whitespace=keep_whitespace)
    eng_words = word_tokenize(text, keep_whitespace=keep_whitespace)
    
    # ตรวจสอบว่าต้องการเก็บตัวอักษรพิเศษหรือไม่
    if not keep_punctuations:
        thai_words = [word for word in thai_words if not is_punct(word)]
        eng_words = [word for word in eng_words if not is_punct(word)]
    
    words = thai_words + eng_words
    return words

def is_punct(word):
    """
    ตรวจสอบว่าเป็นตัวอักษรพิเศษหรือไม่
    """
    return all(char in string.punctuation for char in word)

'''''

'\'\'\nimport string\nimport pythainlp\nfrom pythainlp.tokenize import word_tokenize\n\ndef tokenize(text, keep_whitespace=False, keep_punctuations=False):\n    thai_words = pythainlp.word_tokenize(text, engine=\'newmm\', keep_whitespace=keep_whitespace)\n    eng_words = word_tokenize(text, keep_whitespace=keep_whitespace)\n    \n    # ตรวจสอบว่าต้องการเก็บตัวอักษรพิเศษหรือไม่\n    if not keep_punctuations:\n        thai_words = [word for word in thai_words if not is_punct(word)]\n        eng_words = [word for word in eng_words if not is_punct(word)]\n    \n    words = thai_words + eng_words\n    return words\n\ndef is_punct(word):\n    """\n    ตรวจสอบว่าเป็นตัวอักษรพิเศษหรือไม่\n    """\n    return all(char in string.punctuation for char in word)\n\n'

In [245]:
pythainlp.word_tokenize("12.12 ซื้อ E-Voucher 500บ.ได้600บ.ซื้อเลย!>https://w.ems.to/ef0mjBX")

['12.12',
 ' ',
 'ซื้อ',
 ' ',
 'E-Voucher',
 ' ',
 '500',
 'บ.',
 'ได้',
 '600',
 'บ.',
 'ซื้อ',
 'เลย',
 '!>',
 'https',
 '://',
 'w',
 '.',
 'ems',
 '.',
 'to',
 '/',
 'ef',
 '0',
 'mjBX']

In [246]:
word_tokenize("12.12 ซื้อ E-Voucher 500บ.ได้600บ.ซื้อเลย!>https://w.ems.to/ef0mjBX")

['12.12',
 'ซื้อ',
 'E-Voucher',
 '500บ.ได้600บ.ซื้อเลย',
 '!',
 '>',
 'https',
 ':',
 '//w.ems.to/ef0mjBX']

In [247]:
tokenize("12.12 ซื้อ E-Voucher400 500บ.ได้600บ.ซื้อเลย!>https://w.ems.to/ef0mjBX")

['num',
 '.',
 'num',
 'ซื้อ',
 'E-Vouchernum',
 'num',
 'บ.',
 'ได้',
 'num',
 'บ.',
 'ซื้อ',
 'เลย',
 '!>',
 'https',
 '://',
 'w',
 '.',
 'ems',
 '.',
 'to',
 '/',
 'efnummjBX',
 'num.num',
 'ซื้อ',
 'E-Vouchernum',
 'numบ.ได้numบ.ซื้อเลย',
 '!',
 '>',
 'https',
 ':',
 '//w.ems.to/efnummjBX']

In [248]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

In [249]:
vectorizer = TfidfVectorizer(tokenizer=tokenize)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [250]:
print(X_train_vectorized.shape)
print(X_test_vectorized.shape)

(320, 2191)
(80, 2191)


In [251]:
X_train_vectorized.toarray()

array([[0.16996657, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.13208716, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [252]:
X_test_vectorized.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Naive Bayes


In [253]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


# split data
#X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], random_state=0)

# vectorize data
#vectorizer = CountVectorizer()
#X_train_vectorized = vectorizer.fit_transform(X_train)
#X_test_vectorized = vectorizer.transform(X_test)


# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_vectorized, y_train)
y_pred_nb = nb.predict(X_test_vectorized)

# evaluate model
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print('Accuracy of Naive Bayes classifier:', accuracy_nb)

Accuracy of Naive Bayes classifier: 0.975


In [254]:
X_test_vectorized._shape

(80, 2191)

In [255]:
report = classification_report(y_test, y_pred_nb)
print(report)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97        36
           1       1.00      0.95      0.98        44

    accuracy                           0.97        80
   macro avg       0.97      0.98      0.97        80
weighted avg       0.98      0.97      0.98        80



In [256]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
acc_score = accuracy_score(y_test, y_pred_nb)
conf_mat_nb = confusion_matrix(y_test, y_pred_nb)

print("confusion_matrix Naive Bayes : \n",conf_mat_nb)


confusion_matrix Naive Bayes : 
 [[36  0]
 [ 2 42]]


Random Forest

In [257]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# vectorize data
#vectorizer = CountVectorizer()
#X_train_vectorized = vectorizer.fit_transform(X_train)
#X_test_vectorized = vectorizer.transform(X_test)

# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train_vectorized, y_train)
y_pred_rf = rf.predict(X_test_vectorized)

# evaluate model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print('Accuracy of Random Forest classifier:', accuracy_rf)

Accuracy of Random Forest classifier: 0.9375


In [258]:
from sklearn.metrics import classification_report
report_rf = classification_report(y_test, y_pred_rf)
print(report_rf)


              precision    recall  f1-score   support

           0       0.92      0.94      0.93        36
           1       0.95      0.93      0.94        44

    accuracy                           0.94        80
   macro avg       0.94      0.94      0.94        80
weighted avg       0.94      0.94      0.94        80



In [259]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
acc_score = accuracy_score(y_test, y_pred_rf)
conf_mat_rf = confusion_matrix(y_test, y_pred_rf)

print("confusion_matrix Naive Bayes : \n",conf_mat_rf)

confusion_matrix Naive Bayes : 
 [[34  2]
 [ 3 41]]


SVM

In [260]:
# import libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [261]:
import pythainlp
from nltk.tokenize import word_tokenize

def tokenize(text):
    thai_words = pythainlp.word_tokenize(text, engine='newmm')
    eng_words = word_tokenize(text)
    words = thai_words + eng_words
    return words

In [262]:
# import libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# vectorize data
vectorizer = TfidfVectorizer(tokenizer=tokenize)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# SVM
model_SVM = SVC(kernel='linear')
model_SVM.fit(X_train_vectorized, y_train)

# evaluate model
y_pred_SVM = model_SVM.predict(X_test_vectorized)
accuracy_SVM = accuracy_score(y_test, y_pred_SVM)
print('Accuracy of SVM classifier:', accuracy_SVM)

Accuracy of SVM classifier: 0.9625


In [263]:
from sklearn.metrics import classification_report
report_SVM = classification_report(y_test, y_pred_SVM)
print(report_SVM)


              precision    recall  f1-score   support

           0       0.95      0.97      0.96        36
           1       0.98      0.95      0.97        44

    accuracy                           0.96        80
   macro avg       0.96      0.96      0.96        80
weighted avg       0.96      0.96      0.96        80



In [264]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
acc_score = accuracy_score(y_test, y_pred_SVM)
conf_mat_SVM = confusion_matrix(y_test, y_pred_SVM)

print("confusion_matrix Naive Bayes : \n",conf_mat_SVM)

confusion_matrix Naive Bayes : 
 [[35  1]
 [ 2 42]]


LSTM


In [265]:
import numpy as np
import pandas as pd
import re
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences

from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pythainlp
from nltk.tokenize import word_tokenize

In [266]:
# อ่านข้อมูล SMS จากไฟล์ CSV
#data = pd.read_csv('Data-SMS.csv', encoding='utf-8')

In [267]:
#data = data.rename(columns={'Class': 'label', 'Text': 'text'})
#data['label'] = data['label'].map({'real': 0, 'defraud': 1})

In [268]:
def tokenize(text):
    text = re.sub(r'\d+', 'num', text)
    thai_words = pythainlp.word_tokenize(text,keep_whitespace=False, engine='newmm')
    eng_words = word_tokenize(text)
    #words = [word.replace(' ','' ) for word in thai_words + eng_words]
    words = thai_words + eng_words
    return words

In [269]:
# แปลงข้อมูลเป็นตัวเลขโดยใช้ Tokenizer
tokenizer = Tokenizer(num_words=5000, split=' ')
tokenizer.fit_on_texts(data['text'].apply(lambda x: ' '.join(pythainlp.word_tokenize(x, engine='newmm',keep_whitespace=False)) + ' '.join(word_tokenize(x))).values)
X = tokenizer.texts_to_sequences(data['text'].apply(lambda x: ' '.join(pythainlp.word_tokenize(x, engine='newmm',keep_whitespace=False)) + ' '.join(word_tokenize(x))).values)
X = pad_sequences(X)

In [270]:
# แปลงป้ายกำกับเป็นตัวเลข 0 หรือ 1 (ham หรือ spam)
y = pd.get_dummies(data['label']).values


In [271]:
# แบ่งชุดข้อมูลเป็นชุดสำหรับ Train และ Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [272]:
# กำหนดโมเดล LSTM

model_LSTM = Sequential()
model_LSTM.add(Embedding(5000, 128, input_length=X.shape[1]))
model_LSTM.add(Dropout(0.3))
model_LSTM.add(LSTM(128))
model_LSTM.add(Dropout(0.3))
model_LSTM.add(Dense(2, activation='softmax'))
#model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model_LSTM.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 118, 128)          640000    
                                                                 
 dropout_10 (Dropout)        (None, 118, 128)          0         
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 771,842
Trainable params: 771,842
Non-trainable params: 0
_________________________________________________________________
None


In [273]:
# ฝึกโมเดล
model_LSTM.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
5/5 [==============================] - 4s 435ms/step - loss: 0.6888 - accuracy: 0.6938 - val_loss: 0.6770 - val_accuracy: 0.8875
Epoch 2/10
5/5 [==============================] - 1s 270ms/step - loss: 0.6585 - accuracy: 0.8406 - val_loss: 0.6306 - val_accuracy: 0.7000
Epoch 3/10
5/5 [==============================] - 1s 260ms/step - loss: 0.5683 - accuracy: 0.8594 - val_loss: 0.4797 - val_accuracy: 0.9250
Epoch 4/10
5/5 [==============================] - 1s 266ms/step - loss: 0.3978 - accuracy: 0.9156 - val_loss: 0.3591 - val_accuracy: 0.9500
Epoch 5/10
5/5 [==============================] - 1s 264ms/step - loss: 0.3190 - accuracy: 0.9812 - val_loss: 0.3192 - val_accuracy: 0.9500
Epoch 6/10
5/5 [==============================] - 1s 264ms/step - loss: 0.2390 - accuracy: 0.9844 - val_loss: 0.2328 - val_accuracy: 0.9500
Epoch 7/10
5/5 [==============================] - 1s 273ms/step - loss: 0.1548 - accuracy: 0.9750 - val_loss: 0.1828 - val_accuracy: 0.9500
Epoch 8/10
5/5 [====

In [274]:
from sklearn.metrics import classification_report

# Make predictions on the test set
y_pred = model_LSTM.predict(X_test)

# Convert the predictions to binary values
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

# Generate the classification report
print(classification_report(y_test, y_pred))

3/3 [==============================] - 1s 24ms/step
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        36
           1       0.95      0.95      0.95        44

    accuracy                           0.95        80
   macro avg       0.95      0.95      0.95        80
weighted avg       0.95      0.95      0.95        80



In [275]:
from sklearn.metrics import accuracy_score
accuracy_LSTM = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_LSTM}")

Accuracy: 0.95


In [276]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
acc_score = accuracy_score(y_test, y_pred)
conf_mat_LSTM = confusion_matrix(y_test, y_pred)

print("confusion_matrix Naive Bayes : \n",conf_mat_LSTM)

confusion_matrix Naive Bayes : 
 [[34  2]
 [ 2 42]]


รวม

In [277]:
#การแยกข้อมูลด้วย TfidfVectorizer

print('Accuracy of Naive Bayes  :', accuracy_nb)
print('Accuracy of RandomForest :', accuracy_rf)
print('Accuracy of SVM          :', accuracy_SVM)
print('Accuracy of LSTM         :', accuracy_LSTM)

Accuracy of Naive Bayes  : 0.975
Accuracy of RandomForest : 0.9375
Accuracy of SVM          : 0.9625
Accuracy of LSTM         : 0.95


In [278]:
print("confusion_matrix Naive Bayes : \n",conf_mat_nb)

print("confusion_matrix Random Forest : \n",conf_mat_rf)

print("confusion_matrix SVM  : \n",conf_mat_SVM)

print("confusion_matrix LSTM : \n",conf_mat_LSTM)

confusion_matrix Naive Bayes : 
 [[36  0]
 [ 2 42]]
confusion_matrix Random Forest : 
 [[34  2]
 [ 3 41]]
confusion_matrix SVM  : 
 [[35  1]
 [ 2 42]]
confusion_matrix LSTM : 
 [[34  2]
 [ 2 42]]


wc

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pythainlp
from pythainlp.tokenize import word_tokenize

In [ ]:
sms = pd.read_csv('data_test.csv')
print(sms.head())

In [ ]:
def tokenize(text):
    text = re.sub(r'\d+', 'num', text)
    thai_words = pythainlp.word_tokenize(text,keep_whitespace=False, engine='newmm')
    eng_words = word_tokenize(text)
    #words = [word.replace(' ','' ) for word in thai_words + eng_words]
    words = thai_words + eng_words
    return words

In [ ]:
sms['words'] = sms['sms_text'].apply(tokenize)

In [ ]:
sms['words']

In [ ]:
text = ' '.join([word for words in sms['words'] for word in words])
wordcloud = WordCloud(width=800, height=800,
                      background_color='white',
                      font_path='THSarabunNew.ttf',
                      stopwords=STOPWORDS,
                      min_font_size=10,
                      colormap='Dark2').generate(text)

# แสดง Word Cloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
  
plt.show()
plt.savefig("1.jpg")

In [ ]:
# รวมเป็นข้อความเดียว
all_words = ' '.join([word for words in sms['words'] for word in words])

# สร้าง WordCloud object
wordcloud = WordCloud(font_path='THSarabunNew.ttf',str='black', background_color='white', width=800, height=400, colormap='tab20b').generate(all_words)

# แสดง WordCloud
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
plt.savefig("1.jpg")

In [ ]:
all_words

text

In [ ]:
from pythainlp import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# อ่านข้อมูล SMS จากไฟล์ text
with open("sms.txt", encoding='utf8') as f:
    text = f.read()

# แยกคำที่เป็นภาษาไทยและอังกฤษเท่านั้น
thai_words = [word for word in word_tokenize(text, engine='newmm') if word.isalpha() and word != " "]
eng_words = [word for word in word_tokenize(text) if word.isalpha() and word != " "]

# รวมคำภาษาไทยและอังกฤษเข้าด้วยกัน
words = thai_words + eng_words

# นับความถี่ของคำแต่ละคำ
word_counts = dict()
for word in words:
    if word not in word_counts:
        word_counts[word] = 1
    else:
        word_counts[word] += 1

# สร้าง Word Cloud
wordcloud = WordCloud(background_color="white", width=800, height=400, colormap="Blues", max_words=100).generate_from_frequencies(word_counts)

# แสดง Word Cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
plt.savefig('2.jpg')

In [ ]:
sms = pd.read_csv('data_test.csv')


def tokenize(text):
    thai_words = pythainlp.word_tokenize(text,keep_whitespace=False, engine='newmm')
    return words

sms['words'] = sms['sms_text'].apply(tokenize)

text = ' '.join([word for words in sms['words'] for word in words])
wordcloud = WordCloud(width=800, height=800,
                      background_color='white',
                      font_path='THSarabunNew.ttf',
                      stopwords=STOPWORDS,
                      min_font_size=10,
                      colormap='Dark2').generate(text)

# แสดง Word Cloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
  
plt.show()
plt.savefig("1.jpg")